In [18]:
import requests
import yaml
import re
from tqdm import tqdm
from time import sleep
import time
import os
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute


In [22]:
resp = requests.get(" http://api.steampowered.com/ISteamApps/GetAppList/v0002/?format=json")
app_list = resp.json()
apps = app_list['applist']['apps']

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)


In [23]:
len(apps)


131984

In [25]:
def data_insert(game_info):
    need_data = ['name','steam_appid', 'required_age', 'is_free', 'short_description', 'developers', 'publishers', 'price_overview', 
             'platforms', 'release_date', 'supported_languages', 'categories', 'genres', 'recommendations']
    
    need_data_dic ={}
    for key, data in game_info.items():
        if key in need_data:
            if key in ["developers", "publishers"]:
                data = data[0]

            elif key == "price_overview":
                need_data_dic['final_price'] = data['final']
                need_data_dic['initial_price'] = data['initial']
                need_data_dic['discount_percent'] = data['discount_percent']
                continue

            elif key == "platforms":
                res = ""
                for platform, boolean in data.items():
                    if boolean:
                       res += platform 
                data = res

            elif key == "release_date":
                if data['coming_soon']:
                    data = 'coming_soon'
                else:
                    data = data['date']
            elif key == "supported_languages":
                data = re.sub('<.*?>|[-*]', '',data).replace("음성이 지원되는 언어",", 음성이 지원되는 언어")

            elif key == "categories":
                category_id = []
                category_value = []
                for category_info in data:
                    category_id.append(str(category_info['id']))
                    category_value.append(category_info['description'])
                need_data_dic['categories_id'] = ','.join(category_id)
                need_data_dic['categories'] = ','.join(category_value)
                continue

            elif key == "genres":
                genre_id = []
                genre_value = []
                for genre_info in data:
                    genre_id.append(str(genre_info['id']))
                    genre_value.append(genre_info['description'])
                need_data_dic['genres_id'] = ','.join(genre_id)
                need_data_dic['genres'] = ','.join(genre_value)
                continue

            elif key == "recommendations":
                data = data['total']

            if type(data) is str:
                need_data_dic[key] = re.sub('(<([^>^-]+)>)', '',data)
            else:
                need_data_dic[key] = data
                
    placeholders = ', '.join(['%s'] * len(need_data_dic))
    columns = ', '.join(need_data_dic.keys())
    
    sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('game_info', columns, placeholders)
    
    return db_function.execute_insert(sql, list(need_data_dic.values()))

In [ ]:
start_time = time.time()
try_cnt = 0
db_function = None
try:    
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])
    for app_info in tqdm(apps):
        try_cnt +=1
        current_time = time.time()
        elapsed_time = current_time - start_time

        if try_cnt > 200 and elapsed_time < 300:
            sleep(60)
            try_cnt = 0
            start_time = time.time()

        try:
            appid = app_info['appid']
            sql = "SELECT id FROM game_info WHERE steam_appid=%s" % appid
            if db_function.execute_select(sql):
                continue

            url = f"https://store.steampowered.com/api/appdetails?appids={appid}&l=korean"

            resp = requests.get(url)
            if not resp.ok:
                print(url, resp)

            game_info_resp = resp.json()
            if game_info_resp and game_info_resp[str(appid)]['success']:
                res = data_insert(game_info_resp)
            sleep(5)

        except KeyboardInterrupt as e:
            break

        except:
            raise
            
finally:
    db_function.curs.close()
    db_function.conn.close()

  0%|          | 52/131984 [04:35<194:24:14,  5.30s/it]